In [27]:
# !pip install youtube_transcript_api
# !pip install -U -q google.generativeai
# !pip install streamlit
# !pip install python-dotenv


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from youtube_transcript_api import YouTubeTranscriptApi
import google.generativeai as genai
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

In [18]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [21]:
prompt = """Welcome, Video quiz generator! Your task is to generate a quiz from a given YouTube video transcript, potentially from a video lecture. Your quiz should test on the key points and essential information, presented in 5 multiple choice questions. Let's dive into the provided transcript and generate quiz for our audience."""

In [22]:
def extract_transcript_details(youtube_video_url):
    try:
        video_id = youtube_video_url.split("=")[1]
        transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for i in transcript_text:
            transcript += " " + i["text"]

        return transcript
    except Exception as e:
        raise e

def generate_gemini_content(transcript, prompt):
    model = genai.GenerativeModel("gemini-1.5-pro-latest")
    response = model.generate_content(prompt + transcript)
    return response.text
